In [27]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install textblob
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.1 MB 2.4 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=9530 sha256=de38bbfbcbe55b5031ad0575077adec56dbd19078155d3f413bf126794032634
  Stored in directory: /home/arjun/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c4

Defaulting to user installation because normal site-packages is not writeable


In [20]:
# """
# This code was written by Viraj Shetty and Arjun Praveen!
# """
# import pandas as pd
# import time
# import operator
# import bs4 as bs
# import datetime
# from bs4 import BeautifulSoup
# import urllib.request
# from urllib.request import urlopen
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.firefox.options import Options
# from datetime import date
# from textblob import TextBlob

# """
# This little piece of code is added to make the webdriver run silently
# Generally when you use WebDriver in Selenium it opens a new window
# If you have automated some functions you can also see those functions happening live
# To avoid it use this option called "headless" which does all this in the background

# Web Driver object creation
# Note: Before importing Selenium Webdriver
#     Make sure you have a webdriver application installed in the same directory
    
#     For Mozzilla
#     >https://github.com/mozilla/geckodriver/releases
    
#     For Chrome
#     >https://chromedriver.chromium.org/downloads
# """
# options = Options()
# options.headless = True
# browser = webdriver.Firefox(options = options)


# # This function tokenizes the entered keywords

# def tokenizer(l_t, l_d):
#     v = TextBlob(l_t)
#     w = TextBlob(l_d)
#     a = v.noun_phrases
#     b = w.noun_phrases
#     c = a + b
#     d = ""
#     for i in c:
#         d = d + i + " "
#     print(d)
#     return d

# # Function for extracting number of days since the video has been uploaded

# def number_of_days(v_date):
#     today = date.today()
#     month = {
#              "Jan" : 1,
#              "Feb" : 2,
#              "Mar" : 3,
#              "Apr" : 4,
#              "May" : 5,
#              "Jun" : 6,
#              "Jul" : 7,
#              "Aug" : 8,
#              "Sep" : 9,
#              "Oct" : 10,
#              "Nov" : 11,
#              "Dec" : 12
#             }
#     v=v_date.split(" ")
#     v_year = int(v[2])
#     v_month = int(month[v[0]])
#     v_day = int(v[1])
#     v_date = date(v_year,v_month,v_day)
#     time = abs(today - v_date)
#     return time.days

# def search():
    
#     keyword = input("Enter title: ")
    
#     description = input("Enter description: ")  
    
#     keysearch = tokenizer(keyword, description)
#     keysearch_list = keysearch.split(" ")
#     key = []
#     for j in keysearch_list:
#         if j not in key:
#             key.append(j)
#     print(key)
#     keysearch = ""
#     for k in key:
#         keysearch = keysearch + k + " "
#     print("Scraping search results for the keyword: " + keysearch)
    
#     tag_list = keysearch.split(" ")
#     browser = webdriver.Firefox()
#     browser.get(f'https://www.youtube.com/results?search_query={keysearch}&sp=EgIQAQ%253D%253D')
#     browser.maximize_window()

#     """
#     Please note here you need to use an Adblock to avoid advertisement videos to be a part of your list
#     Add the filepath to your .xpi file
#     """
#     extension = "{your }" 
#     #browser.install_addon(extension, temporary = True)

#     # Change value of While Loop to scroll more pages
#     i=0
#     while i<=1:
#         html = browser.find_element_by_tag_name('html')
#         '''
#         The code in the next line takes the page to end. Since in YouTube if you want more
#         videos to load, you have to scroll down to the end of the page
#         '''
#         html.send_keys(Keys.END)
#         time.sleep(3)
#         i+=1
        
#     v_l = browser.find_elements_by_xpath('//*[@id="video-title"]')
#     link = []
#     for i in v_l:
#         link.append(i.get_attribute('href'))
        
#     #print(link) 
#     wait = WebDriverWait(browser,15)
#     df = pd.DataFrame(columns = ['Link','Title','URL','Views','Likes','Dislikes','Subcount','Date','Days','Score','Tags'])
#     k=0
#     for x in link:
#         try:
#             i = 0
#             browser.get(x)
#             time.sleep(3)
#             v_link = x
#             v_title = wait.until(EC.presence_of_element_located((By.XPATH,'//h1[@class="title style-scope ytd-video-primary-info-renderer"]'))).text
#             v_views = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/div[5]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/yt-view-count-renderer/span[1]'))).text
#             try:    
#                 v_views = int(v_views.strip(' views').replace(',', '', 2))
#             except:
#                 v_views = 1
#             v_likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.ytd-menu-renderer:nth-child(1) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
#             v_dislikes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.style-scope:nth-child(2) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            
#             #if (v_likes == 0 & v_dislikes == 0):
#                 #v_like_dislike = 0
            
#             #Likes
#             try:
#                 if (operator.contains(v_likes,'K')):
#                     if(operator.contains(v_likes,'.')):
#                         pos = len(v_likes)-v_likes.find('.')-1 #length from the end
#                         a1 = v_likes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
#                         v_likes = num
#                     else:
#                         v_likes=int(v_likes[ :len(v_likes)-1])*(1000)
#                 elif operator.contains(v_likes,'M'):
#                     if(operator.contains(v_likes,'.')):
#                         pos = len(v_likes)-v_likes.find('.')-1 #length from the end
#                         a1 = v_likes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
#                         v_likes = num
#                     else:
#                         v_likes=int(v_likes[ :len(v_likes)-1])*(1000000)
#                 else:
#                     v_likes = int(v_likes)
#             except:
#                 v_likes = 1

#             #Dislikes
#             try:
#                 if (operator.contains(v_dislikes,'K')):
#                     if(operator.contains(v_dislikes,'.')):
#                         pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
#                         a1 = v_dislikes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
#                         v_dislikes = num
#                     else:
#                         v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000)
#                 elif operator.contains(v_dislikes,'M'):
#                     if(operator.contains(v_dislikes,'.')):
#                         pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
#                         a1 = v_dislikes.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
#                         v_dislikes = num
#                     else:
#                         v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000000)
#                 else:
#                     v_dislikes = int(v_dislikes)
#             except:
#                 v_dislikes = 1
            
#             v_like_dislike = ((v_likes-v_dislikes)/v_likes)
            
#             v_subcount = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="owner-sub-count"]'))).text
#             v_subcount = v_subcount.strip(" subscribers")
#             try:
#                 if operator.contains(v_subcount,'M'):
#                     if(operator.contains(v_subcount,'.')):
#                         pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
#                         a1 = v_subcount.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
#                         v_subcount = num
#                     else:
#                         v_subcount=int(v_subcount[ :len(v_subcount)-1])*(1000000)
#                 elif operator.contains(v_subcount,'K'):
#                     if(operator.contains(v_subcount,'.')):
#                         pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
#                         a1 = v_subcount.replace('.', '', 1)
#                         num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
#                         v_subcount = num
#                     else:
#                         v_subcount = int(v_subcount[ :len(v_subcount)-1])*(1000)
#                 else:
#                     v_subcount = int(float(v_subcount))
#             except:
#                 v_subcount = 1
#             v_urls = browser.find_elements_by_xpath('//div[@class="ytp-cued-thumbnail-overlay-image"]')
#             for i in v_urls:
#                 v_url=i.get_attribute('style')[23:-3]
#                 break
#             v_date = browser.find_element_by_xpath("//div[@id='date']").text
#             if(operator.contains(v_date,'Premiered')):
#                 v_date = v_date[10:]
#             elif(operator.contains(v_date,'Streamed live on')):
#                 v_date = v_date[17:]

#             v_date = v_date[1:].replace(",","",1)
#             v_days = number_of_days(v_date)
        
# #             try:
# #                 url = x
# #                 html_content = urlopen(url)
# #                 soup = BeautifulSoup(html_content,'lxml')
# #                 v_tags = [tag["content"] for tag in soup.findAll("meta",property="og:video:tag")]
# #             except:
# #                 print('Error')
# #             co = 0
            
# #             for tl in tag_list:
# #                 for t in v_tags:
# #                     if operator.contains(t, tl):
# #                         co+=1
# #             v_tag = co/(len(tag_list))
            
#             # Score formula
#             try:
#                 v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike) +(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) + (12 * v_tag))/43
#             except:
#                 v_days = 1
#                 v_likes = 1
#                 v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike)+(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) + (12 * v_tag))/43
            
            
#             print('{}: {}/{} ,{} , {:.2f}'.format(url,co,len(tag_list),v_tag,v_score))

            
#             df.loc[len(df)] = [v_link,v_title,v_url,v_views,v_likes,v_dislikes,v_subcount,v_date,v_days,v_score,v_tags]
#             k
#             k+=1
#             print(k)
#         #Exception added since some YouTube ads bypass the adblock
#         except Exception as e:
#             print(e)
#             continue
#     df = df.sort_values(by='Score', ascending = 0)
#     return df
#     browser.close()

# n = int(input("How many Links would you like to print? "))
# df1 = search()
# df = df1.nlargest(n,'Score')
# print(df)

How many Links would you like to print? 5
Enter title: Python
Enter description: 
python 
['python', '']
Scraping search results for the keyword: python  
Message: invalid type: null, expected a string at line 1 column 12

'Started'
name 'v_tag' is not defined
name 'v_tag' is not defined
name 'v_tag' is not defined


TypeError: Column 'Score' has dtype object, cannot use method 'nlargest' with this dtype

In [24]:
"""
This code was written by Viraj Shetty and Arjun Praveen!
"""
import pandas as pd
import time
import operator
import bs4 as bs
import datetime
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from datetime import date
from textblob import TextBlob

"""
This little piece of code is added to make the webdriver run silently
Generally when you use WebDriver in Selenium it opens a new window
If you have automated some functions you can also see those functions happening live
To avoid it use this option called "headless" which does all this in the background

Web Driver object creation
Note: Before importing Selenium Webdriver
    Make sure you have a webdriver application installed in the same directory
    
    For Mozzilla
    >https://github.com/mozilla/geckodriver/releases
    
    For Chrome
    >https://chromedriver.chromium.org/downloads
"""
options = Options()
options.headless = True
browser = webdriver.Firefox(options = options)


# This function tokenizes the entered keywords

def tokenizer(l_t, l_d):
    v = TextBlob(l_t)
    w = TextBlob(l_d)
    a = v.noun_phrases
    b = w.noun_phrases
    c = a + b
    d = ""
    for i in c:
        d = d + i + " "
    print(d)
    return d

# Function for extracting number of days since the video has been uploaded

def number_of_days(v_date):
    today = date.today()
    month = {
             "Jan" : 1,
             "Feb" : 2,
             "Mar" : 3,
             "Apr" : 4,
             "May" : 5,
             "Jun" : 6,
             "Jul" : 7,
             "Aug" : 8,
             "Sep" : 9,
             "Oct" : 10,
             "Nov" : 11,
             "Dec" : 12
            }
    v=v_date.split(" ")
    v_year = int(v[2])
    v_month = int(month[v[0]])
    v_day = int(v[1])
    v_date = date(v_year,v_month,v_day)
    time = abs(today - v_date)
    return time.days

def search():
    
    keyword = input("Enter title: ")
    
    description = input("Enter description: ")  
    
    keysearch = tokenizer(keyword, description)
    keysearch_list = keysearch.split(" ")
    key = []
    for j in keysearch_list:
        if j not in key:
            key.append(j)
    print(key)
    keysearch = ""
    for k in key:
        keysearch = keysearch + k + " "
    print("Scraping search results for the keyword: " + keysearch)
    
    tag_list = keysearch.split(" ")
    browser = webdriver.Firefox()
    browser.get(f'https://www.youtube.com/results?search_query={keysearch}&sp=EgIQAQ%253D%253D')
    browser.maximize_window()

    """
    Please note here you need to use an Adblock to avoid advertisement videos to be a part of your list
    Add the filepath to your .xpi file
    """
    extension = "{your }" 
    #browser.install_addon(extension, temporary = True)

    # Change value of While Loop to scroll more pages
    i=0
    while i<=1:
        html = browser.find_element_by_tag_name('html')
        '''
        The code in the next line takes the page to end. Since in YouTube if you want more
        videos to load, you have to scroll down to the end of the page
        '''
        html.send_keys(Keys.END)
        time.sleep(3)
        i+=1
        
    v_l = browser.find_elements_by_xpath('//*[@id="video-title"]')
    link = []
    for i in v_l:
        link.append(i.get_attribute('href'))
        
    #print(link) 
    wait = WebDriverWait(browser,15)
    df = pd.DataFrame(columns = ['Link','Title','URL','Views','Likes','Dislikes','Subcount','Date','Days','Score'])
    k=0
    for x in link:
        try:
            i = 0
            browser.get(x)
            time.sleep(3)
            v_link = x
            v_title = wait.until(EC.presence_of_element_located((By.XPATH,'//h1[@class="title style-scope ytd-video-primary-info-renderer"]'))).text
            v_views = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/div[5]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/yt-view-count-renderer/span[1]'))).text
            try:    
                v_views = int(v_views.strip(' views').replace(',', '', 2))
            except:
                v_views = 1
            v_likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.ytd-menu-renderer:nth-child(1) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            v_dislikes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.style-scope:nth-child(2) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            
            #if (v_likes == 0 & v_dislikes == 0):
                #v_like_dislike = 0
            
            #Likes
            try:
                if (operator.contains(v_likes,'K')):
                    if(operator.contains(v_likes,'.')):
                        pos = len(v_likes)-v_likes.find('.')-1 #length from the end
                        a1 = v_likes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_likes = num
                    else:
                        v_likes=int(v_likes[ :len(v_likes)-1])*(1000)
                elif operator.contains(v_likes,'M'):
                    if(operator.contains(v_likes,'.')):
                        pos = len(v_likes)-v_likes.find('.')-1 #length from the end
                        a1 = v_likes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_likes = num
                    else:
                        v_likes=int(v_likes[ :len(v_likes)-1])*(1000000)
                else:
                    v_likes = int(v_likes)
            except:
                v_likes = 1

            #Dislikes
            try:
                if (operator.contains(v_dislikes,'K')):
                    if(operator.contains(v_dislikes,'.')):
                        pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
                        a1 = v_dislikes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_dislikes = num
                    else:
                        v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000)
                elif operator.contains(v_dislikes,'M'):
                    if(operator.contains(v_dislikes,'.')):
                        pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
                        a1 = v_dislikes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_dislikes = num
                    else:
                        v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000000)
                else:
                    v_dislikes = int(v_dislikes)
            except:
                v_dislikes = 1
            
            v_like_dislike = ((v_likes-v_dislikes)/v_likes)
            
            v_subcount = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="owner-sub-count"]'))).text
            v_subcount = v_subcount.strip(" subscribers")
            try:
                if operator.contains(v_subcount,'M'):
                    if(operator.contains(v_subcount,'.')):
                        pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
                        a1 = v_subcount.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_subcount = num
                    else:
                        v_subcount=int(v_subcount[ :len(v_subcount)-1])*(1000000)
                elif operator.contains(v_subcount,'K'):
                    if(operator.contains(v_subcount,'.')):
                        pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
                        a1 = v_subcount.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_subcount = num
                    else:
                        v_subcount = int(v_subcount[ :len(v_subcount)-1])*(1000)
                else:
                    v_subcount = int(float(v_subcount))
            except:
                v_subcount = 1
            v_urls = browser.find_elements_by_xpath('//div[@class="ytp-cued-thumbnail-overlay-image"]')
            for i in v_urls:
                v_url=i.get_attribute('style')[23:-3]
                break
            v_date = browser.find_element_by_xpath("//div[@id='date']").text
            if(operator.contains(v_date,'Premiered')):
                v_date = v_date[10:]
            elif(operator.contains(v_date,'Streamed live on')):
                v_date = v_date[17:]

            v_date = v_date[1:].replace(",","",1)
            v_days = number_of_days(v_date)
            url = x
            co =0
#             try:
#                 url = x
#                 html_content = urlopen(url)
#                 soup = BeautifulSoup(html_content,'lxml')
#                 v_tags = [tag["content"] for tag in soup.findAll("meta",property="og:video:tag")]
#             except:
#                 print('Error')
#             co = 0
            
#             for tl in tag_list:
#                 for t in v_tags:
#                     if operator.contains(t, tl):
#                         co+=1
#             v_tag = co/(len(tag_list))
            
            # Score formula
            try:
                v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike) +(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200))/31
            except:
                v_days = 1
                v_likes = 1
                v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike)+(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) )/31
            
            
            print('{}:  {:.2f}'.format(url, v_score))

            
            df.loc[len(df)] = [v_link,v_title,v_url,v_views,v_likes,v_dislikes,v_subcount,v_date,v_days,v_score]
            k
            k+=1
            print(k)
        #Exception added since some YouTube ads bypass the adblock
        except Exception as e:
            #print(e)
            continue
    df = df.sort_values(by='Score', ascending = 0)
    return df
    browser.close()

n = int(input("How many Links would you like to print? "))
df1 = search()
df = df1.nlargest(n,'Score')
print(df)

How many Links would you like to print? 5
Enter title: Python
Enter description: 
python 
['python', '']
Scraping search results for the keyword: python  
Message: invalid type: null, expected a string at line 1 column 12

https://www.youtube.com/watch?v=_uQrJ0TkZlc:  62.85
1
https://www.youtube.com/watch?v=rfscVS0vtbw:  75.69
2
https://www.youtube.com/watch?v=WGJJIrtnfpk:  10.09
3
'Streamed'
https://www.youtube.com/watch?v=4F2m91eKmts:  14.76
4
https://www.youtube.com/watch?v=kLZuut1fYzQ:  16.24
5
https://www.youtube.com/watch?v=pJ3IPRqiD2M:  16.15
6
https://www.youtube.com/watch?v=eTyI-M50Hu4:  6.33
7
https://www.youtube.com/watch?v=ihk_Xglr164:  8.04
8
https://www.youtube.com/watch?v=OnDr4J2UXSA:  7.95
9
https://www.youtube.com/watch?v=Y8Tko2YC5hA:  8.49
10
https://www.youtube.com/watch?v=r-uOLxNrNk8:  7.11
11
https://www.youtube.com/watch?v=Km2UZ5bQTvQ:  3.95
12
https://www.youtube.com/watch?v=J0Aq44Pze-w:  5.41
13
https://www.youtube.com/watch?v=edvg4eHi_Mw:  6.98
14
https://www.y

In [26]:
df1

,Link,Title,URL,Views,Likes,Dislikes,Subcount,Date,Days,Score
16,https://www.youtube.com/watch?v=JgKN3BuvC3E,"Python, Honey Badger & Jackal Fight Each Other",https://i.ytimg.com/vi_webp/JgKN3BuvC3E/maxres...,24316080,104000,6500.0,57300.0,Dec 9 2019,146,99.569129
1,https://www.youtube.com/watch?v=rfscVS0vtbw,Learn Python - Full Course for Beginners [Tuto...,https://i.ytimg.com/vi_webp/rfscVS0vtbw/maxres...,13930413,329000,3800.0,1910000.0,Jul 11 2018,662,75.685381
0,https://www.youtube.com/watch?v=_uQrJ0TkZlc,Python Tutorial for Beginners [Full Course] Le...,https://i.ytimg.com/vi_webp/_uQrJ0TkZlc/maxres...,9487486,304000,2300.0,1020000.0,Feb 18 2019,440,62.853457
23,https://www.youtube.com/watch?v=iE7BpqtfgYA,Python is too aggressive - Big Cat Can't Escap...,https://i.ytimg.com/vi_webp/iE7BpqtfgYA/maxres...,14711587,39000,11000.0,1.0,Jul 18 2019,290,40.618762
19,https://www.youtube.com/watch?v=rbasThWVb-c,Creating a Snake game with Python in under 5 m...,https://i.ytimg.com/vi_webp/rbasThWVb-c/maxres...,2263897,116000,1800.0,314000.0,Jun 10 2017,1058,19.820407
34,https://www.youtube.com/watch?v=4JF6Uh0Xyrs,"Python, Anaconda FIGHT over LIVE Squirrel -- E...",https://i.ytimg.com/vi_webp/4JF6Uh0Xyrs/maxres...,7306599,32000,9200.0,452000.0,Jun 1 2019,337,16.379453
4,https://www.youtube.com/watch?v=kLZuut1fYzQ,What Can You Do with Python? - The 3 Main Appl...,https://i.ytimg.com/vi_webp/kLZuut1fYzQ/maxres...,2131678,61000,926.0,1320000.0,Jun 22 2018,681,16.244865
5,https://www.youtube.com/watch?v=pJ3IPRqiD2M,Python Course | Python Tutorial for Beginners ...,https://i.ytimg.com/vi_webp/pJ3IPRqiD2M/maxres...,1801860,65000,1200.0,488000.0,Aug 16 2019,261,16.145195
3,https://www.youtube.com/watch?v=4F2m91eKmts,Python Tutorial for Beginners - Full Course in...,https://i.ytimg.com/vi_webp/4F2m91eKmts/maxres...,972074,63000,1000.0,570000.0,Dec 21 2019,134,14.762969
15,https://www.youtube.com/watch?v=8DvywoWv6fI,Python for Everybody - Full Course with Dr. Chuck,https://i.ytimg.com/vi_webp/8DvywoWv6fI/maxres...,1094740,36000,317.0,1910000.0,May 2 2019,367,11.900110


In [41]:
import plotly.graph_objects as go

colors = ['lightslategray',] * 5

for i in range(len):
    colors[i] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=[i for i in range(len(df1))],
    y=[i for i in df1.Likes],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text='Likes per video')





In [34]:
a = [x for x in range(len(df1))]

b = [x for x in df1.Likes]
b

[104000,
 329000,
 304000,
 39000,
 116000,
 32000,
 61000,
 65000,
 63000,
 36000,
 29000,
 20000,
 20000,
 12000,
 20000,
 13000,
 9300.0,
 15000,
 14000,
 12000,
 3600.0,
 8600.0,
 7900.0,
 13000,
 10000,
 7600.0,
 4100.0,
 4800.0,
 1700.0,
 4500.0,
 1,
 168,
 383,
 1,
 3300.0,
 4100.0,
 1200.0,
 1300.0,
 1300.0,
 4900.0]